# Get error function
Difference between empirical data an simulated

Last update: 26/02/2018

In [1]:
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import os
import random
import re
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
data_f='./data/'
fitbit = pd.read_csv(data_f+'fitbit.csv', sep=';', header=0)

In [33]:
fitbit.head()

,School,Class,Wave,Child_Bosse,Day,Gender,Age,Steps,Minutes_MVPA,Steps_ML_imp1,Minutes_MVPA_ML_imp1,imputed,Start,Finish,Date
0,22,52.0,1,648,5,1,12,NaN,NaN,NaN,NaN,NaN,2016-01-27,2016-02-02,2016-02-01
1,22,52.0,1,643,2,1,13,NaN,NaN,7624.0,31.0,1.0,2016-01-27,2016-02-02,2016-01-29
2,22,52.0,1,643,1,1,13,NaN,NaN,13617.0,70.0,1.0,2016-01-27,2016-02-02,2016-01-28
3,22,52.0,1,650,3,0,12,3320.0,0.0,3320.0,0.0,0.0,2016-01-27,2016-02-02,2016-01-30
4,22,52.0,1,643,5,1,13,NaN,NaN,NaN,NaN,NaN,2016-01-27,2016-02-02,2016-02-01


In [53]:
# Range of dates Wave 1 (Start)
pd.to_datetime(fitbit[fitbit.Wave==1].Start, format='%Y-%m-%d').min(), \
pd.to_datetime(fitbit[fitbit.Wave==1].Start, format='%Y-%m-%d').max()

(Timestamp('2016-01-26 00:00:00'), Timestamp('2016-03-04 00:00:00'))

In [57]:
# Range of dates Wave 2
pd.to_datetime(fitbit[fitbit.Wave==2].Start, format='%Y-%m-%d').min(), \
pd.to_datetime(fitbit[fitbit.Wave==2].Start, format='%Y-%m-%d').max()

(Timestamp('2016-03-30 00:00:00'), Timestamp('2016-05-12 00:00:00'))

In [58]:
# Range of dates Wave 3
pd.to_datetime(fitbit[fitbit.Wave==3].Start, format='%Y-%m-%d').min(), \
pd.to_datetime(fitbit[fitbit.Wave==3].Start, format='%Y-%m-%d').max()

(Timestamp('2016-05-31 00:00:00'), Timestamp('2016-06-17 00:00:00'))

In [59]:
# Range of dates Wave 4
pd.to_datetime(fitbit[fitbit.Wave==4].Start, format='%Y-%m-%d').min(), \
pd.to_datetime(fitbit[fitbit.Wave==4].Start, format='%Y-%m-%d').max()

(Timestamp('2017-02-14 00:00:00'), Timestamp('2017-03-23 00:00:00'))

In [4]:
fitbit.groupby(['Wave', 'Day']).count()

School  Class  Child_Bosse  Gender  Age  Steps  Minutes_MVPA  \
Wave Day                                                                 
1    1       953    927          953     953  953    518           518   
     2       953    927          953     953  953    447           447   
     3       953    927          953     953  953    395           395   
     4       953    927          953     953  953    309           309   
     5       953    927          953     953  953    174           174   
2    1       953    927          953     953  953    608           608   
     2       953    927          953     953  953    544           544   
     3       953    927          953     953  953    468           468   
     4       953    927          953     953  953    393           393   
     5       953    927          953     953  953    167           167   
3    1       953    927          953     953  953    476           476   
     2       953    927          953     953  953    401           401   
     3       953    927          953     953  953    323           323   
     4       953    927          953     953  953    252           252   
     5       953    927          953     953  953    118           118   
4    1       953    927          953     953  953    503           503   
     2       953    927          953     953  953    418           418   
     3       953    927          953     953  953    331           331   
     4       953    927          953     953  953    276           276   
     5       953    927          953     953  953    148           148   

          Steps_ML_imp1  Minutes_MVPA_ML_imp1  imputed  Start  Finish  Date  
Wave Day                                                                     
1    1              953                   953      953    927     927   927  
     2              953                   953      953    927     927   927  
     3              953                   953      953    927     927   927  
     4              309                   309      309    927     927   927  
     5              174                   174      174    927     927   927  
2    1              953                   953      953    927     927   927  
     2              953                   953      953    927     927   927  
     3              953                   953      953    927     927   927  
     4              393                   393      393    927     927   927  
     5              167                   167      167    927     927   927  
3    1              953                   953      953    927     927   927  
     2              953                   953      953    927     927   927  
     3              953                   953      953    927     927   927  
     4              252                   252      252    927     927   927  
     5              118                   118      118    927     927   927  
4    1              953                   953      953    913     913   913  
     2              953                   953      953    913     913   913  
     3              953                   953      953    913     913   913  
     4              276                   276      276    913     913   913  
     5              148                   148      148    913     913   913

# PA

The current calculation is based on steps.

The value of the steps were changed based on the following assumptions:

* 1.53 is a normal level for sedentary kids
* 10.000 steps is the match for the sedentary kids

So we multiplied the steps by 1.53/10.000 to get the values for the "normalized" PA.

In [74]:
# reset_index to convert the groupby multiindex to a column 
# (https://stackoverflow.com/questions/20110170/turn-pandas-multi-index-into-column)
steps_mean_wave = fitbit.groupby(['Child_Bosse', 'Wave']).mean()['Steps_ML_imp1'].reset_index()
steps_mean_wave.head()

,Child_Bosse,Wave,Steps_ML_imp1
0,643,1,8868.000000
1,643,2,6997.000000
2,643,3,10799.333333
3,643,4,7500.333333
4,645,1,10747.400000


In [75]:
steps_mean_wave.Steps_ML_imp1 = steps_mean_wave.Steps_ML_imp1 * 0.000153

In [76]:
steps_mean_wave[steps_mean_wave.Wave==1].head()

,Child_Bosse,Wave,Steps_ML_imp1
0,643,1,1.356804
4,645,1,1.644352
8,648,1,0.711807
12,649,1,1.199469
16,650,1,1.534988


In [77]:
steps_mean_wave = steps_mean_wave.pivot(index='Child_Bosse', columns='Wave')['Steps_ML_imp1']

In [79]:
steps_mean_wave[1]

Child_Bosse
643     1.356804
645     1.644352
648     0.711807
649     1.199469
650     1.534988
666     1.475302
676     0.934779
677     0.976038
683     1.587120
685     1.177845
688     1.714403
690     1.657857
693     1.915162
694     1.255314
698     1.577940
721     1.657908
733     1.328295
736     2.348907
737     1.461380
738     1.809123
739     1.268829
744     2.157963
745     0.883652
747     1.279488
761     1.989867
766     1.823505
767     1.379203
768     1.127916
770     2.573358
771     1.315596
          ...   
3057    1.392249
3058    2.082381
4103    1.787295
4104    1.339821
4105    1.189575
4106    1.695699
4107    1.652859
4108    1.524135
4119    1.672341
4121    1.506693
4152    1.083189
4155    1.875015
4163    1.679481
4166    0.899436
4167    1.639395
4168    1.379907
4190    0.840429
4614    2.708355
4616    2.025516
4618    0.705330
4621    1.247715
4622    0.686511
4625    1.085943
4626    1.036575
4627    1.876749
4628    1.169379
4629    1.134291
46